In [158]:
import pandas as pd
import matplotlib as plt
import numpy as np
from sqlite3 import connect

| x1 | x2 |
| -- | -- |
| info | info 2 | 

In [159]:
df = pd.read_csv('vgchartz-2024.csv')

In [160]:
df.head()

,img,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
3,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
4,/games/boxart/full_4990510AmericaFrontccc.jpg,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14


In [161]:
df['release_date'] = pd.to_datetime(df['release_date'])

df.drop(['img', 'last_update'], axis=1, inplace=True)

df = df.dropna(subset=['release_date'])

columns_to_fill_zero = ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales']

df[columns_to_fill_zero] = df[columns_to_fill_zero].fillna(0)



In [162]:
df.columns 

Index(['title', 'console', 'genre', 'publisher', 'developer', 'critic_score',
       'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales',
       'release_date'],
      dtype='object')

In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56965 entries, 0 to 64014
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   title         56965 non-null  object        
 1   console       56965 non-null  object        
 2   genre         56965 non-null  object        
 3   publisher     56965 non-null  object        
 4   developer     56954 non-null  object        
 5   critic_score  56965 non-null  float64       
 6   total_sales   56965 non-null  float64       
 7   na_sales      56965 non-null  float64       
 8   jp_sales      56965 non-null  float64       
 9   pal_sales     56965 non-null  float64       
 10  other_sales   56965 non-null  float64       
 11  release_date  56965 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(6), object(5)
memory usage: 5.6+ MB


In [164]:
df_yrs = df.sort_values(by='release_date', ascending=True)
df_yrs

,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date
30650,The Oregon Trail,Series,Education,MECC,MECC,0.0,0.00,0.00,0.0,0.00,0.00,1971-12-03
57351,Trek73,PC,Simulation,Unknown,"William K. Char, Perry Lee, and Dan Gee",0.0,0.00,0.00,0.0,0.00,0.00,1973-10-08
56948,Nintengirls: College Girl & Friends,DS,Simulation,Nintendo,Nintendo EAD,0.0,0.00,0.00,0.0,0.00,0.00,1975-10-12
1532,Air-Sea Battle,2600,Shooter,Atari,Atari,0.0,0.98,0.91,0.0,0.06,0.01,1977-01-01
20694,Fun With Numbers,2600,Action,Atari,Atari,0.0,0.00,0.00,0.0,0.00,0.00,1977-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
57500,Cities: Skylines II,PS5,Simulation,Paradox Interactive,Colossal Order,0.0,0.00,0.00,0.0,0.00,0.00,2024-12-01
57501,Cities: Skylines II,XS,Simulation,Paradox Interactive,Colossal Order,0.0,0.00,0.00,0.0,0.00,0.00,2024-12-01
55377,Judas,PS5,Shooter,Ghost Story Games,Ghost Story Games,0.0,0.00,0.00,0.0,0.00,0.00,2024-12-31
55379,Judas,PC,Shooter,Ghost Story Games,Ghost Story Games,0.0,0.00,0.00,0.0,0.00,0.00,2024-12-31


# exp

In [165]:
df['console_mfg'] = np.select(
    [
        df['console'].isin(['3DS', 'ds', 'Wii', 'WiiU', 'NS', 'GB', 'NES', 'SNES', 'GBC', 'N64', 'ds']),
        df['console'].isin(['Linux', 'OSX', 'PC', 'Arc', 'All', 'Series']),
        df['console'].isin(['X360', 'XOne']),
        df['console'].isin(['PS', 'PS2', 'PS3', 'PS4', 'PS5', 'PSP', 'PSV', 'PSN']),
        df['console'].isin(['iOS', 'And']),
        df['console'].isin(['GG', 'MSD', 'MS', 'GEN', 'SCD']),
        df['console'].isin(['2600', '7800']),
        df['console'].isin(['amig', 'C64', 'amig']),
        df['console'].isin(['Ouya', 'OR', 'ACPC', 'AST', 'ApII', 'PCE', 'ZXS', 'Lynx', 'NG', 'ZXS'])
    ],
    [
        'nintendo',
        'pc',
        'xbox',
        'sony',
        'mobile',
        'sega',
        'atari',
        'commodore',
        'other'
    ],
    default='unknown'
)

df.head(20)

,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,console_mfg
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,sony
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,sony
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,sony
3,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,0.0,15.86,9.06,0.06,5.33,1.42,2013-09-17,xbox
4,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,sony
5,Call of Duty: Modern Warfare 3,X360,Shooter,Activision,Infinity Ward,8.7,14.82,9.07,0.13,4.29,1.33,2011-11-08,xbox
6,Call of Duty: Black Ops,X360,Shooter,Activision,Treyarch,8.8,14.74,9.76,0.11,3.73,1.14,2010-11-09,xbox
7,Red Dead Redemption 2,PS4,Action-Adventure,Rockstar Games,Rockstar Games,9.8,13.94,5.26,0.21,6.21,2.26,2018-10-26,sony
8,Call of Duty: Black Ops II,X360,Shooter,Activision,Treyarch,8.4,13.86,8.27,0.07,4.32,1.20,2012-11-13,xbox
9,Call of Duty: Black Ops II,PS3,Shooter,Activision,Treyarch,8.0,13.80,4.99,0.65,5.88,2.28,2012-11-13,sony


In [166]:
# Original lists (updated to lowercase)
nintendo = ['3DS', 'ds', 'Wii', 'WiiU', 'NS', 'GB', 'NES', 'SNES', 'GBC', 'N64']
pc = ['Linux', 'OSX', 'PC', 'Arc', 'All', 'Series']
xbox = ['X360', 'XOne']
sony = ['PS', 'PS2', 'PS3', 'PS4', 'PS5', 'PSP', 'PSV', 'PSN']
mobile = ['iOS', 'And']
sega = ['GG', 'MSD', 'MS', 'GEN', 'SCD']
atari = ['2600', '7800']
commodore = ['amig', 'C64']
other = ['Ouya', 'OR', 'ACPC', 'AST', 'ApII', 'PCE', 'ZXS', 'Lynx', 'NG', 'ZXS']

# Convert all items to lowercase and strip whitespaces
all_items_lower = [item.lower().strip() for item in nintendo + pc + xbox + sony + mobile + sega + atari + commodore + other]

# Unique values in the 'console' column of your DataFrame (converted to lowercase and stripped)
unique_console_values_lower = df['console'].str.lower().str.strip().unique()

# Check if any items are missing
missing_items = set(all_items_lower) - set(unique_console_values_lower)

if missing_items:
    print(f"Missing items: {missing_items}")
else:
    print("All items are covered.")


All items are covered.


In [167]:
def check_missing_items(all_items, df_column):
    all_items_lower = [item.lower().strip() for item in all_items]
    unique_values_lower = set(df_column.str.lower().str.strip().unique())
    return set(all_items_lower) - unique_values_lower

def assign_console_mfg(df):
    conditions = [
        df['console'].isin(['3DS', 'ds', 'Wii', 'WiiU', 'NS', 'GB', 'NES', 'SNES', 'GBC', 'N64', 'ds']),
        df['console'].isin(['Linux', 'OSX', 'PC', 'Arc', 'All', 'Series']),
        df['console'].isin(['X360', 'XOne']),
        df['console'].isin(['PS', 'PS2', 'PS3', 'PS4', 'PS5', 'PSP', 'PSV', 'PSN']),
        df['console'].isin(['iOS', 'And']),
        df['console'].isin(['GG', 'MSD', 'MS', 'GEN', 'SCD']),
        df['console'].isin(['2600', '7800']),
        df['console'].isin(['amig', 'C64', 'amig']),
        df['console'].isin(['Ouya', 'OR', 'ACPC', 'AST', 'ApII', 'PCE', 'ZXS', 'Lynx', 'NG', 'ZXS'])
    ]

    values = ['nintendo', 'pc', 'xbox', 'sony', 'mobile', 'sega', 'atari', 'commodore', 'other']
    
    df['console_mfg'] = np.select(conditions, values, default='unknown')


# Original lists
nintendo = ['3DS', 'ds', 'Wii', 'WiiU', 'NS', 'GB', 'NES', 'SNES', 'GBC', 'N64']
pc = ['Linux', 'OSX', 'PC', 'Arc', 'All', 'Series']
xbox = ['X360', 'XOne']
sony = ['PS', 'PS2', 'PS3', 'PS4', 'PS5', 'PSP', 'PSV', 'PSN']
mobile = ['iOS', 'And']
sega = ['GG', 'MSD', 'MS', 'GEN', 'SCD']
atari = ['2600', '7800']
commodore = ['amig', 'C64']
other = ['Ouya', 'OR', 'ACPC', 'AST', 'ApII', 'PCE', 'ZXS', 'Lynx', 'NG', 'ZXS']

# Combined list of all items
all_items = nintendo + pc + xbox + sony + mobile + sega + atari + commodore + other

# Checking for missing items
missing_items = check_missing_items(all_items, df['console'])

if missing_items:
    print(f"Missing items: {missing_items}")
else:
    print("All items are covered.")

# Assignment of values to 'console_mfg' column
assign_console_mfg(df)

# Display the DataFrame with the new 'console_mfg' column
df.head(20)


All items are covered.


,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,console_mfg
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,sony
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,sony
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,sony
3,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,0.0,15.86,9.06,0.06,5.33,1.42,2013-09-17,xbox
4,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,sony
5,Call of Duty: Modern Warfare 3,X360,Shooter,Activision,Infinity Ward,8.7,14.82,9.07,0.13,4.29,1.33,2011-11-08,xbox
6,Call of Duty: Black Ops,X360,Shooter,Activision,Treyarch,8.8,14.74,9.76,0.11,3.73,1.14,2010-11-09,xbox
7,Red Dead Redemption 2,PS4,Action-Adventure,Rockstar Games,Rockstar Games,9.8,13.94,5.26,0.21,6.21,2.26,2018-10-26,sony
8,Call of Duty: Black Ops II,X360,Shooter,Activision,Treyarch,8.4,13.86,8.27,0.07,4.32,1.20,2012-11-13,xbox
9,Call of Duty: Black Ops II,PS3,Shooter,Activision,Treyarch,8.0,13.80,4.99,0.65,5.88,2.28,2012-11-13,sony


# corr

In [168]:
def check_missing_items(all_items, df_column):
    all_items_lower = [item.lower().strip() for item in all_items]
    unique_values_lower = set(df_column.str.lower().str.strip().unique())
    return set(all_items_lower) - unique_values_lower

def assign_console_mfg(df):
    categories = {
        'nintendo': ['3DS', 'ds', 'Wii', 'WiiU', 'NS', 'GB', 'NES', 'SNES', 'GBC', 'N64'],
        'pc': ['Linux', 'OSX', 'PC', 'Arc', 'All', 'Series'],
        'xbox': ['X360', 'XOne'],
        'sony': ['PS', 'PS2', 'PS3', 'PS4', 'PS5', 'PSP', 'PSV', 'PSN'],
        'mobile': ['iOS', 'And'],
        'sega': ['GG', 'MSD', 'MS', 'GEN', 'SCD'],
        'atari': ['2600', '7800'],
        'commodore': ['amig', 'C64'],
        'other': ['Ouya', 'OR', 'ACPC', 'AST', 'ApII', 'PCE', 'ZXS', 'Lynx', 'NG', 'ZXS']
    }

    all_items = [item for sublist in categories.values() for item in sublist]

    # Checking for missing items
    missing_items = check_missing_items(all_items, df['console'])

    if missing_items:
        print(f"Missing items: {missing_items}")
    else:
        print("All items are covered.")

    # Assignment of values to 'console_mfg' column
    conditions = [df['console'].isin(items) for items in categories.values()]
    values = list(categories.keys())

    df['console_mfg'] = np.select(conditions, values, default='unknown')


# Assignment of values to 'console_mfg' column and checking for missing items
assign_console_mfg(df)

# Display the DataFrame with the new 'console_mfg' column
df.head(20)


All items are covered.


,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,console_mfg
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,sony
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,sony
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,sony
3,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,0.0,15.86,9.06,0.06,5.33,1.42,2013-09-17,xbox
4,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,sony
5,Call of Duty: Modern Warfare 3,X360,Shooter,Activision,Infinity Ward,8.7,14.82,9.07,0.13,4.29,1.33,2011-11-08,xbox
6,Call of Duty: Black Ops,X360,Shooter,Activision,Treyarch,8.8,14.74,9.76,0.11,3.73,1.14,2010-11-09,xbox
7,Red Dead Redemption 2,PS4,Action-Adventure,Rockstar Games,Rockstar Games,9.8,13.94,5.26,0.21,6.21,2.26,2018-10-26,sony
8,Call of Duty: Black Ops II,X360,Shooter,Activision,Treyarch,8.4,13.86,8.27,0.07,4.32,1.20,2012-11-13,xbox
9,Call of Duty: Black Ops II,PS3,Shooter,Activision,Treyarch,8.0,13.80,4.99,0.65,5.88,2.28,2012-11-13,sony


# SQL Connection

In [169]:
conn = connect(':memory:')

df.to_sql("df", conn)

def sql(a_string):
    return(pd.read_sql(a_string, conn))

Top 20 video games by sales 

In [170]:
sql("""
    SELECT
        title,
        genre,
        publisher,
        developer,
        AVG(critic_score) AS avg_critic_score,
        SUM(total_sales) AS total_sales,
        MIN(release_date) AS release_date
    FROM
        df
    GROUP BY
        title
    ORDER BY
        total_sales DESC
    LIMIT 20 
""")

,title,genre,publisher,developer,avg_critic_score,total_sales,release_date
0,Grand Theft Auto V,Action,Rockstar Games,Rockstar North,3.512500,64.29,2013-09-17 00:00:00
1,Call of Duty: Black Ops,Shooter,Activision,Treyarch,8.287500,30.99,2010-11-09 00:00:00
2,Call of Duty: Modern Warfare 3,Shooter,Activision,Infinity Ward,6.000000,30.71,2011-11-08 00:00:00
3,Call of Duty: Black Ops II,Shooter,Activision,Treyarch,6.700000,29.59,2012-11-13 00:00:00
4,Call of Duty: Ghosts,Shooter,Activision,Infinity Ward,6.714286,28.80,2013-11-05 00:00:00
5,Call of Duty: Black Ops 3,Shooter,Activision,Treyarch,1.620000,26.72,2015-11-06 00:00:00
6,Call of Duty: Modern Warfare 2,Shooter,Activision,Infinity Ward,4.666667,25.02,2009-11-10 00:00:00
7,Minecraft,Misc,Mojang,Mojang AB,3.138462,24.01,2010-05-10 00:00:00
8,Grand Theft Auto IV,Action,Rockstar Games,Rockstar North,5.840000,22.53,2008-04-29 00:00:00
9,Call of Duty: Advanced Warfare,Shooter,Activision,Sledgehammer Games,8.720000,21.78,2014-11-04 00:00:00


Top 20 publishers 

In [171]:
sql("""
    SELECT
        publisher,
        AVG(critic_score) AS avg_critic_score,
        SUM(total_sales) AS total_sales
    FROM
        df
    GROUP BY
        publisher
    ORDER BY
        total_sales DESC
    LIMIT 20
""")

,publisher,avg_critic_score,total_sales
0,Activision,1.652878,722.77
1,Electronic Arts,2.116316,644.13
2,EA Sports,2.121891,485.66
3,Ubisoft,1.615120,462.57
4,THQ,1.503694,320.89
5,Sony Computer Entertainment,1.621501,311.08
6,Rockstar Games,3.310000,239.67
7,Konami,1.014971,210.70
8,Sega,1.056593,206.38
9,Nintendo,2.570508,140.80


Top 20 Developers

In [172]:
sql("""
    SELECT
        developer,
        AVG(critic_score) AS avg_critic_score,
        SUM(total_sales) AS total_sales,
        MIN(release_date) AS release_date
    FROM
        df
    GROUP BY
        developer
    ORDER BY
        total_sales DESC
    LIMIT 20 
""")

,developer,avg_critic_score,total_sales,release_date
0,EA Canada,2.339130,275.56,1992-01-01 00:00:00
1,EA Tiburon,2.875299,178.33,1995-01-01 00:00:00
2,Ubisoft Montreal,2.825461,172.96,2001-10-15 00:00:00
3,Treyarch,3.874737,150.19,1998-02-23 00:00:00
4,Traveller's Tales,2.238647,149.55,1992-01-01 00:00:00
5,Infinity Ward,3.983673,118.97,2003-10-29 00:00:00
6,Rockstar North,4.736364,116.91,1998-03-27 00:00:00
7,Konami,0.701737,109.42,1981-01-01 00:00:00
8,Visual Concepts,2.206422,97.61,1991-10-01 00:00:00
9,Capcom,1.500899,92.95,1984-12-01 00:00:00


Top Genre

In [173]:
sql("""
    SELECT
        genre,
        AVG(critic_score) AS avg_critic_score,
        SUM(total_sales) AS total_sales,
        MIN(release_date) AS release_date
    FROM
        df
    GROUP BY
        Genre
    ORDER BY
        total_sales DESC
    LIMIT 20 
""")

,genre,avg_critic_score,total_sales,release_date
0,Sports,0.937914,1186.77,1977-01-01 00:00:00
1,Action,0.944825,1124.95,1977-01-01 00:00:00
2,Shooter,1.324475,995.47,1977-01-01 00:00:00
3,Misc,0.263793,557.55,1977-01-01 00:00:00
4,Racing,1.171696,523.51,1977-01-01 00:00:00
5,Role-Playing,1.315013,425.88,1979-01-01 00:00:00
6,Platform,1.154443,349.15,1978-01-01 00:00:00
7,Fighting,1.138357,340.39,1980-01-01 00:00:00
8,Adventure,0.540130,325.09,1977-01-01 00:00:00
9,Simulation,0.564616,300.48,1973-10-08 00:00:00


Top 5 video games with the highest total sales within each console group

In [174]:
sql("""
    SELECT
        console,
        title,
        genre,
        publisher,
        developer,
        critic_score,
        total_sales
    FROM (
        SELECT
            console,
            title,
            genre,
            publisher,
            developer,
            critic_score,
            total_sales,
            ROW_NUMBER() OVER (PARTITION BY console ORDER BY total_sales DESC) AS rank
        FROM
            df
    ) ranked
    WHERE
        rank <= 5
""")

,console,title,genre,publisher,developer,critic_score,total_sales
0,2600,Asteroids,Shooter,Atari,Atari,0.0,4.31
1,2600,Demon Attack,Shooter,Imagic,Imagic,0.0,2.13
2,2600,E.T.: The Extra Terrestrial,Action,Atari,Atari,0.0,1.97
3,2600,Ms. Pac-Man,Puzzle,Atari,General Entertainment,0.0,1.65
4,2600,Dig Dug,Puzzle,Atari,Atari,0.0,1.64
...,...,...,...,...,...,...,...
369,iOS,Super Mario Run,Action,Nintendo,Nintendo,0.0,0.00
370,iOS,CounterSpy,Action,Sony Computer Entertainment,Dynamighty,0.0,0.00
371,iOS,PlayStation All-Stars Island,Action,Sony Computer Entertainment,Zoink Games,0.0,0.00
372,iOS,Bentley's Hackpack,Action,Sony Computer Entertainment,Sony Computer Entertainment,0.0,0.00


total sales for each combination of console and year

In [175]:
sql("""
    SELECT
        console,
        strftime('%Y', release_date) AS year,
        SUM(total_sales) AS total_sales
    FROM
        df
    GROUP BY
        console,
        year
    ORDER BY
        year, console
""")

,console,year,total_sales
0,Series,1971,0.0
1,PC,1973,0.0
2,DS,1975,0.0
3,2600,1977,2.5
4,PC,1977,0.0
...,...,...,...
866,PC,2024,0.0
867,PS4,2024,0.0
868,PS5,2024,0.0
869,XOne,2024,0.0


In [176]:
sql("""
    SELECT
        console,
        strftime('%Y', release_date) AS year,
        SUM(total_sales) AS total_sales
    FROM
        df
    WHERE
        strftime('%Y', release_date) = '2016'
    GROUP BY
        console,
        year
    ORDER BY
     year, console;
""")

,console,year,total_sales
0,3DS,2016,10.72
1,All,2016,0.00
2,And,2016,0.00
3,Arc,2016,0.00
4,DC,2016,0.00
5,DS,2016,0.02
6,Linux,2016,0.00
7,OR,2016,0.00
8,OSX,2016,0.00
9,Ouya,2016,0.00


In [177]:
sql("""
    WITH RankedGames AS (
    SELECT
        console,
        title,
        total_sales,
        ROW_NUMBER() OVER (PARTITION BY console ORDER BY total_sales DESC) AS rank
    FROM
        df
    WHERE
        strftime('%Y', release_date) = '1999'
)
SELECT
    console,
    title,
    total_sales
FROM
    RankedGames
WHERE
    rank = 1;
""")

,console,title,total_sales
0,AJ,Protector,0.00
1,All,Asterix & Obelix Take On Caesar,0.00
2,Amig,WipEout 2097,0.00
3,DC,Sonic Adventure,2.42
4,GB,Medarot 2: Kabuto / Kuwagata Version,0.54
5,GBC,Pokémon Pinball,0.00
6,Lynx,Lexis,0.00
7,N64,Namco Museum 64,1.45
8,NG,Bakumatsu Rouman Dai Ni Maku: Gekka no Kenshi ...,0.00
9,OSX,StarCraft,0.00


In [178]:
nintendo = [ 3DS, ds, Wii, WiiU, NS, GB, NES, SNES, GBC, N64]
pc = [ Linux, OSX, PC, Arc, All, Series ]
xbox = [x360, XOne,]
sony = [ PS, PS2, PS3, PS4, PS5, PSP, PSV, PSN, ]
mobile = [ iOS, And, ]
sega = [GG, MSD, MS, GEN, SCD]
atari = [2600, 7800 ]
commodore = [ amig, C64, ]
other = [ Ouya, OR, ACPC, AST, ApII, PCE, ZXS, Lynx, NG, ZXS ]

SyntaxError: invalid decimal literal (4133319350.py, line 1)

In [ ]:
df['release_date'] = pd.to_datetime(df['release_date'])
df_yr = df[df['release_date'].dt.year == 2019]  
result = df_yr.groupby(['release_date', 'console'])['total_sales'].sum().reset_index()
result['release_date'] = result['release_date'].dt.year  
result = result.groupby(['release_date', 'console'])['total_sales'].sum().reset_index()

result

,release_date,console,total_sales
0,2019,3DS,1.03
1,2019,All,0.00
2,2019,And,0.00
3,2019,Linux,0.00
4,2019,NS,0.27
5,2019,OR,0.00
6,2019,OSX,0.00
7,2019,PC,0.02
8,2019,PS4,1.23
9,2019,PSV,0.00


Defines a function called `filter_and_group_by_year` 
- which takes a DataFrame (df) and a target year as input
 
- It filters the DataFrame to include only rows from the specified year

- Groups the data by 'release_date' and 'console'
 
- calculates the sum of 'total_sales' for each group
 
- Updates the 'release_date' column to show only the year.

- Then, groups the data again by the modified 'release_date' and 'console' columns

- Calculates the sum of 'total_sales' again 

- Returns the resulting DataFrame.

In the second part of the code
- there is a loop that iterates through years from 1971 to 2023
- In each iteration, it dynamically creates a DataFrame with a name like 'sales_1971', 'sales_1972', etc. 
- Then assigns the result of the function for that specific year to the newly created DataFrame variable.

This loop essentially generates DataFrames for years 1971 - 2023, 
<br>
each representing the sales data for a particular year, and assigns
<br>
them unique names using the 'sales_year' format.

In [ ]:
def filter_and_group_by_year(df, target_year):
    df['release_date'] = pd.to_datetime(df['release_date'])
    df_filtered = df[df['release_date'].dt.year == target_year]
    result = df_filtered.groupby(['release_date', 'console'])['total_sales'].sum().reset_index()
    result['release_date'] = result['release_date'].dt.year  
    result = result.groupby(['release_date', 'console'])['total_sales'].sum().reset_index()
    return result

for year in range(1971, 2024):
    df_name = f"sales_{year}"
    globals()[df_name] = filter_and_group_by_year(df, year)

In [ ]:
sales_1988

,release_date,console,total_sales
0,1988,2600,1.58
1,1988,7800,0.00
2,1988,ACPC,0.00
3,1988,AST,0.00
4,1988,All,0.00
5,1988,Amig,0.00
6,1988,C64,0.00
7,1988,GEN,0.00
8,1988,Int,0.00
9,1988,MS,0.00
